## Test Conference
This conference is based on the test_iclr_conference_v2.py, with some simplifications

Some notes: 
- Some code cannot be run more than once without giving an error (for example if you withdraw a paper, then withdraw it again, it will give you an error)
- Most code depends on all of the code before it being run, since this is simulating a compressed conference workflow

In [22]:
from helpers import *
import time
import openreview
import pytest
import datetime
import re
import random
import os
import csv
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException
selenium = webdriver.Firefox()
helpers = Helpers()

## Make sample users and create deadline
From: test_create_conference()

This section connects to the client, sets up variables for deadlines, and makes sample profiles for the conference

In [23]:
#Set up deadlines
now = datetime.datetime.utcnow()
abstract_date = now + datetime.timedelta(days=1)
due_date = now + datetime.timedelta(days=3)

#connect to client
client = openreview.Client(baseurl = 'http://localhost:3000', username = 'openreview.net', password=helpers.strong_password)
openreview_client =openreview.Client(baseurl = 'http://localhost:3001', username = 'openreview.net', password=helpers.strong_password)

In [24]:
# Make PCs
helpers.create_user('pc@mail.com', 'Program','Chair')
pc_client = openreview.Client(username='pc@mail.com', password=helpers.strong_password)

# Make SACs, ACs, Reviewers, and one test author
helpers.create_user('sac10@mail.com', 'SAC', 'One')
helpers.create_user('sac2@mail.com', 'SAC', 'Two')

#Make ACs
helpers.create_user('ac1@mail.com', 'AC', 'One')
helpers.create_user('ac2@mail.com', 'AC', 'Two')

#Make Reviewers
helpers.create_user('reviewer1@mail.com', 'Reviewer', 'One')
helpers.create_user('reviewer2@mail.com', 'Reviewer', 'Two')

#Make Author
helpers.create_user('author@mail.com', 'Author', 'One') # Author


Look at a profile using this url: http://localhost:3030/profile?email=[EMAIL]

Example: http://localhost:3030/profile?email=pc@mail.com

All profiles use the same password: Or$3cur3P@ssw0rd (also stored in helpers.strong_password)

## Set up conference
From: test_create_conference()

This section sets up the venue request form and deploys it with the settings for the venue. 

In [25]:
#Post the venue request form and deploty the conference
request_form_note = pc_client.post_note(openreview.Note(
        invitation='openreview.net/Support/-/Request_Form',
        signatures=['~Program_Chair1'],
        readers=[
            'openreview.net/Support',
            '~Program_Chair1'
        ],
        writers=[],
        content={
            'title': 'Test Conference', 
            'Official Venue Name': 'Test Conference', 
            'Abbreviated Venue Name': 'TC 2024',  
            'Official Website URL': 'https//:testconference.com', 
            'program_chair_emails': ['pc@mail.com'],
            'contact_email': 'pc@mail.com', 
            'publication_chairs':'No, our venue does not have Publication Chairs',
            'Area Chairs (Metareviewers)': 'Yes, our venue has Area Chairs',
            'senior_area_chairs': 'No, our venue does not have Senior Area Chairs',
            'Venue Start Date': '2024/07/01',
            'abstract_registration_deadline': abstract_date.strftime('%Y/%m/%d'),
            'Submission Deadline': due_date.strftime('%Y/%m/%d'),
            'Location': 'Virtual',
            'submission_reviewer_assignment': 'Automatic',
            'Author and Reviewer Anonymity': 'Double-blind',
            'reviewer_identity': ['Program Chairs', 'Assigned Senior Area Chair', 'Assigned Area Chair', 'Assigned Reviewers'],
            'area_chair_identity': ['Program Chairs', 'Assigned Senior Area Chair', 'Assigned Area Chair', 'Assigned Reviewers'],
            'senior_area_chair_identity': ['Program Chairs', 'Assigned Senior Area Chair', 'Assigned Area Chair', 'Assigned Reviewers'],
            'Open Reviewing Policy': 'Submissions and reviews should both be public.',
            'submission_readers': 'Everyone (submissions are public)',
            'withdrawn_submissions_visibility': 'Yes, withdrawn submissions should be made public.',
            'withdrawn_submissions_author_anonymity': 'Yes, author identities of withdrawn submissions should be revealed.',
            'desk_rejected_submissions_visibility':'Yes, desk rejected submissions should be made public.',
            'desk_rejected_submissions_author_anonymity':'Yes, author identities of desk rejected submissions should be revealed.',
            'How did you hear about us?': 'ML conferences',
            'Expected Submissions': '100',
            'use_recruitment_template': 'Yes',
            'api_version': '2',
            'submission_license': ['CC BY 4.0', 'CC BY-SA 4.0', 'CC0 1.0'] # Allow authors to select license
        }))

helpers.await_queue()

# Deploy the conference

client.post_note(openreview.Note(
    content={'venue_id': 'TC/2024/Conference'},
    forum=request_form_note.forum,
    invitation='openreview.net/Support/-/Request{}/Deploy'.format(request_form_note.number),
    readers=['openreview.net/Support'],
    referent=request_form_note.forum,
    replyto=request_form_note.forum,
    signatures=['openreview.net/Support'],
    writers=['openreview.net/Support']
))

helpers.await_queue()


The venue will be shown here: http://localhost:3030/

## Reviewer Recruitment Stage
From: test_reviewer_recruitment()

This section sets up and runs reviewer recruitment and shows a sample response from a reviewer

In [26]:
# Set up reviewer recruitment
pc_client=openreview.Client(username='pc@mail.com', password=helpers.strong_password)
request_form=pc_client.get_notes(invitation='openreview.net/Support/-/Request_Form')[0]

reviewer_details = '''reviewer1@mail.com, Reviewer One\nreviewer2@mail.com, Reviewer Two'''
pc_client.post_note(openreview.Note(
    content={
        'title': 'Recruitment',
        'invitee_role': 'Reviewers',
        'invitee_details': reviewer_details,
        'allow_accept_with_reduced_load': 'Yes',
        'invitee_reduced_load': ["1", "2", "3"],
        'invitation_email_subject': 'Invitation to serve as {{invitee_role}}',
        'invitation_email_content': 'Dear {{fullname}},\n\nYou have been nominated to serve as {{invitee_role}}.\n\n{{invitation_url}}\n\nCheers!\n\nProgram Chairs'
    },
    forum=request_form.forum,
    replyto=request_form.forum,
    invitation='openreview.net/Support/-/Request{}/Recruitment'.format(request_form.number),
    readers=['TC/2024/Conference/Program_Chairs', 'openreview.net/Support'],
    signatures=['~Program_Chair1'],
    writers=[]
))

helpers.await_queue()

In [27]:
#example response: accept invitation with reduced load
messages = openreview_client.get_messages(subject = 'Invitation to serve as Reviewer')

for message in messages:
    text = message['content']['text']
    invitation_url = re.search('https://.*\n', text).group(0).replace('https://openreview.net', 'http://localhost:3030').replace('&amp;', '&')[:-1]
    print(invitation_url)
    helpers.respond_invitation(selenium, request_page, invitation_url, accept=True, quota=3)

helpers.await_queue_edit(openreview_client, invitation='TC/2024/Conference/Reviewers/-/Recruitment', count=2)


http://localhost:3030/invitation?id=TC/2024/Conference/Reviewers/-/Recruitment&user=reviewer1%40mail.com&key=f9f3bcfda5e52ce7a79c536ab7c0a66b8b1b9cc96161b9fbc09a06b7de61586e
http://localhost:3030/invitation?id=TC/2024/Conference/Reviewers/-/Recruitment&user=reviewer2%40mail.com&key=6dbbb139634a4c21a702ff2657eb44d916edb3b00f6d8756f73f0c32998859ed


## Make some submissions

From: test_submissions()

Sets up some sample submissions with the user author@mail.com + one coauthor with different domains. There are 12 sample submissions created.



In [28]:
#get the author's client
test_client = openreview.api.OpenReviewClient(token=test_client().token)
request_form=client.get_notes(invitation='openreview.net/Support/-/Request_Form')[0]


#post test submissions
domains = ['umass.edu', 'amazon.com', 'fb.com', 'cs.umass.edu', 'google.com', 'mit.edu', 'deepmind.com', 'co.ux', 'apple.com', 'nvidia.com']
for i in range(1,12):
    note = openreview.api.Note(
        license = 'CC BY-SA 4.0',
        content = {
            'title': { 'value': 'Paper title ' + str(i) },
            'abstract': { 'value': 'This is an abstract ' + str(i) },
            'authorids': { 'value': ['~SomeFirstName_User1', 'author@mail.com', 'andrew@' + domains[i % 10]] },
            'authors': { 'value': ['SomeFirstName User', 'First Last', 'Andrew Mc'] },
            'keywords': { 'value': ['machine learning', 'nlp'] },
            'pdf': {'value': '/pdf/' + 'p' * 40 +'.pdf' },
        }
    )

    test_client.post_note_edit(invitation='TC/2024/Conference/-/Submission',
        signatures=['~SomeFirstName_User1'],
        note=note)

helpers.await_queue_edit(openreview_client, invitation='TC/2024/Conference/-/Submission', count=11)


## Close abstract submission

From: test_post_submission

This section uses the Submission stage to close abstract submission- the field abstract_registration_deadline is updated to now - 28 minutes. 

In [29]:
pc_client=openreview.Client(username='pc@mail.com', password=helpers.strong_password)
request_form=pc_client.get_notes(invitation='openreview.net/Support/-/Request_Form')[0]
venue = openreview.get_conference(client, request_form.id, support_user='openreview.net/Support')

## close abstract submission
#update deadline to now
now = datetime.datetime.utcnow()
abstract_date = now - datetime.timedelta(minutes=28)
due_date = now + datetime.timedelta(days=3)       

pc_client.post_note(openreview.Note(
    content={
        'title': 'Test Conference',
        'Official Venue Name': 'Test Conference',
        'Abbreviated Venue Name': 'TC 2024',
        'Official Website URL': 'https://testconference.com',
        'program_chair_emails': ['pc@mail.com'],
        'contact_email': 'pc@mail.com',
        'publication_chairs':'No, our venue does not have Publication Chairs',
        'Venue Start Date': '2024/07/01',
        'abstract_registration_deadline': abstract_date.strftime('%Y/%m/%d'),
        'Submission Deadline': due_date.strftime('%Y/%m/%d'),
        'Location': 'Virtual',
        'submission_reviewer_assignment': 'Automatic',
        'How did you hear about us?': 'ML conferences',
        'Expected Submissions': '100',
    },
    forum=request_form.forum,
    invitation='openreview.net/Support/-/Request{}/Revision'.format(request_form.number),
    readers=['TC/2024/Conference/Program_Chairs', 'openreview.net/Support'],
    referent=request_form.forum,
    replyto=request_form.forum,
    signatures=['~Program_Chair1'],
    writers=[]
))

helpers.await_queue()
helpers.await_queue_edit(openreview_client, 'TC/2024/Conference/-/Post_Submission-0-1', count=2)
helpers.await_queue_edit(openreview_client, 'TC/2024/Conference/-/Withdrawal-0-1', count=1)
helpers.await_queue_edit(openreview_client, 'TC/2024/Conference/-/Desk_Rejection-0-1', count=1)

## Close full paper submission
From: test_post_submission

The Submission stage is run a second time to close full paper submission- this time the submission deadline is updated.

In [30]:
## close full paper submission
now = datetime.datetime.utcnow()
abstract_date = now - datetime.timedelta(days=2)
due_date = now - datetime.timedelta(minutes=28)        
pc_client.post_note(openreview.Note(
    content={
        'title': 'Test Conference',
        'Official Venue Name': 'Test Conference',
        'Abbreviated Venue Name': 'TC 2024',
        'Official Website URL': 'https://testconference.com',
        'program_chair_emails': ['pc@mail.com'],
        'contact_email': 'pc@mail.com',
        'publication_chairs':'No, our venue does not have Publication Chairs',
        'Venue Start Date': '2024/07/01',
        'abstract_registration_deadline': abstract_date.strftime('%Y/%m/%d'),
        'Submission Deadline': due_date.strftime('%Y/%m/%d'),
        'Location': 'Virtual',
        'submission_reviewer_assignment': 'Automatic',
        'How did you hear about us?': 'ML conferences',
        'Expected Submissions': '100',
    },
    forum=request_form.forum,
    invitation='openreview.net/Support/-/Request{}/Revision'.format(request_form.number),
    readers=['TC/2024/Conference/Program_Chairs', 'openreview.net/Support'],
    referent=request_form.forum,
    replyto=request_form.forum,
    signatures=['~Program_Chair1'],
    writers=[]
))

helpers.await_queue()
helpers.await_queue_edit(openreview_client, 'TC/2024/Conference/-/Post_Submission-0-1', count=3)
helpers.await_queue_edit(openreview_client, 'TC/2024/Conference/-/Withdrawal-0-1', count=2)
helpers.await_queue_edit(openreview_client, 'TC/2024/Conference/-/Desk_Rejection-0-1', count=2)

## Withdraw a submission / Reverse withdrawal
From: test_submission_withdrawal

In [31]:
#withdraw a paper
test_client = openreview.api.OpenReviewClient(token=test_client.token)

withdraw_note = test_client.post_note_edit(invitation='TC/2024/Conference/Submission11/-/Withdrawal',
                            signatures=['TC/2024/Conference/Submission11/Authors'],
                            note=openreview.api.Note(
                                content={
                                    'withdrawal_confirmation': { 'value': 'I have read and agree with the venue\'s withdrawal policy on behalf of myself and my co-authors.' },
                                }
                            ))

helpers.await_queue_edit(openreview_client, edit_id=withdraw_note['id'])

note = test_client.get_note(withdraw_note['note']['forum'])

helpers.await_queue_edit(openreview_client, invitation='TC/2024/Conference/-/Withdrawn_Submission')

pc_openreview_client = openreview.api.OpenReviewClient(username='pc@mail.com', password=helpers.strong_password)



In [32]:
# reverse withdrawal
withdrawal_reversion_note = pc_openreview_client.post_note_edit(invitation='TC/2024/Conference/Submission11/-/Withdrawal_Reversion',
                            signatures=['TC/2024/Conference/Program_Chairs'],
                            note=openreview.api.Note(
                                content={
                                    'revert_withdrawal_confirmation': { 'value': 'We approve the reversion of withdrawn submission.' },
                                }
                            ))

helpers.await_queue_edit(openreview_client, edit_id=withdrawal_reversion_note['id'])
helpers.await_queue_edit(openreview_client, invitation='TC/2024/Conference/Submission11/-/Withdrawal_Reversion')



## Desk reject a paper
From: test_submission_withdrawal

This allows you to reject a paper before it goes through review

In [33]:

#desk-reject paper
desk_reject_note = pc_openreview_client.post_note_edit(invitation=f'TC/2024/Conference/Submission11/-/Desk_Rejection',
                            signatures=['TC/2024/Conference/Program_Chairs'],
                            note=openreview.api.Note(
                                content={
                                    'desk_reject_comments': { 'value': 'Wrong format.' },
                                }
                            ))

helpers.await_queue_edit(openreview_client, edit_id=desk_reject_note['id'])
helpers.await_queue_edit(openreview_client, invitation='TC/2024/Conference/-/Desk_Rejected_Submission')


## Review Stage
from: test_review_stage

This section sets up the review stage by setting up the stage and posting a test review

In [34]:

openreview_client = openreview.api.OpenReviewClient(baseurl = 'http://localhost:3001', username='openreview.net', password=Helpers.strong_password)
## add reviewers to group that didn't accept the invite
openreview_client.add_members_to_group('TC/2024/Conference/Submission1/Reviewers', ['~Reviewer_One1', '~Reviewer_Two1'])


#set up review stage
now = datetime.datetime.utcnow()
due_date = now + datetime.timedelta(days=3)
pc_client=openreview.Client(username='pc@mail.com', password=helpers.strong_password)
request_form=pc_client.get_notes(invitation='openreview.net/Support/-/Request_Form')[0]
review_stage_note=pc_client.post_note(openreview.Note(
    content={
        'review_deadline': due_date.strftime('%Y/%m/%d'),
        'make_reviews_public': 'No, reviews should NOT be revealed publicly when they are posted',
        'release_reviews_to_authors': 'No, reviews should NOT be revealed when they are posted to the paper\'s authors',
        'release_reviews_to_reviewers': 'Review should not be revealed to any reviewer, except to the author of the review',
        'email_program_chairs_about_reviews': 'No, do not email program chairs about received reviews',
    },
    forum=request_form.forum,
    invitation='openreview.net/Support/-/Request{}/Review_Stage'.format(request_form.number),
    readers=['TC/2024/Conference/Program_Chairs', 'openreview.net/Support'],
    replyto=request_form.forum,
    referent=request_form.forum,
    signatures=['~Program_Chair1'],
    writers=[]
))
helpers.await_queue()

In [35]:
 #post a test review
invitation = openreview_client.get_invitation('TC/2024/Conference/Submission1/-/Official_Review')

reviewer_client=openreview.api.OpenReviewClient(username='reviewer1@mail.com', password=helpers.strong_password)

anon_groups = reviewer_client.get_groups(prefix='TC/2024/Conference/Submission1/Reviewer_', signatory='~Reviewer_One1')
anon_group_id = anon_groups[0].id

review_edit = reviewer_client.post_note_edit(
    invitation='TC/2024/Conference/Submission1/-/Official_Review',
    signatures=[anon_group_id],
    note=openreview.api.Note(
        content={
            'title': { 'value': 'Good paper, accept'},
            'review': { 'value': 'Excellent paper, accept'},
            'rating': { 'value': 10},
            'confidence': { 'value': 5},
        }
    )
)

helpers.await_queue_edit(openreview_client, edit_id=review_edit['id'])  

## Comment stage
from: test_comment_stage

The comment stage lets users comment on a paper's forum page, allowing for dicsussion between reviewers, for example, and is often enabled along with the review stage.

In [36]:
#start the comment stage
pc_client=openreview.Client(username='pc@mail.com', password=helpers.strong_password)
pc_client_v2=openreview.api.OpenReviewClient(username='pc@mail.com', password=helpers.strong_password)
request_form=pc_client.get_notes(invitation='openreview.net/Support/-/Request_Form')[0]

# Post an official comment stage note
now = datetime.datetime.utcnow()
start_date = now - datetime.timedelta(days=2)
end_date = now + datetime.timedelta(days=3)
comment_stage_note = pc_client.post_note(openreview.Note(
    content={
        'commentary_start_date': start_date.strftime('%Y/%m/%d'),
        'commentary_end_date': end_date.strftime('%Y/%m/%d'),
        'participants': ['Program Chairs', 'Assigned Senior Area Chairs', 'Assigned Area Chairs', 'Assigned Reviewers'],
        'additional_readers': ['Program Chairs', 'Assigned Senior Area Chairs', 'Assigned Area Chairs', 'Assigned Reviewers', 'Assigned Submitted Reviewers'],
        'email_program_chairs_about_official_comments': 'Yes, email PCs for each official comment made in the venue'
    },
    forum=request_form.forum,
    invitation=f'openreview.net/Support/-/Request{request_form.number}/Comment_Stage',
    readers=['TC/2024/Conference/Program_Chairs', 'openreview.net/Support'],
    replyto=request_form.forum,
    referent=request_form.forum,
    signatures=['~Program_Chair1'],
    writers=[]
))

## Decision stage
from: test_camera_ready_revision_stage

After the review deadline has passed. This section programmatically accepts/declines invitations and puts that in a csv that is uploaded and used to send decisions. Then the decision stage is run.

Note, this makes the decisions, then the post decision stage is what sends notifications

In [37]:
## Programmatically accept / reject papers

pc_client=openreview.Client(username='pc@mail.com', password=helpers.strong_password)
request_form=pc_client.get_notes(invitation='openreview.net/Support/-/Request_Form')[0]

decisions = ['Accept (Oral)', 'Accept (Poster)', 'Reject']
comment = {
    'Accept (Oral)': 'Congratulations on your acceptance.',
    'Accept (Poster)': 'Congratulations on your acceptance.',
    'Reject': 'We regret to inform you...'
}

submissions = openreview_client.get_notes(invitation='TC/2024/Conference/-/Submission', sort='number:asc')

with open(os.path.join(os.path.abspath(''), 'data/ICML_decisions.csv'), 'w') as file_handle:
    writer = csv.writer(file_handle)
    writer.writerow([submissions[0].number, 'Accept (Oral)', comment["Accept (Oral)"]])
    writer.writerow([submissions[1].number, 'Accept (Poster)', comment["Accept (Poster)"]])
    writer.writerow([submissions[2].number, 'Reject', comment["Reject"]])
    for submission in submissions[3:]:
        decision = random.choice(decisions)
        writer.writerow([submission.number, decision, comment[decision]])

decision_stage_invitation = f'openreview.net/Support/-/Request{request_form.number}/Decision_Stage'
url = pc_client.put_attachment(os.path.join(os.path.abspath(''), 'data/ICML_decisions.csv'),
                                    decision_stage_invitation, 'decisions_file')


In [38]:
## start decision stage
now = datetime.datetime.utcnow()
end_date = now + datetime.timedelta(days=3)
decision_stage_note = pc_client.post_note(openreview.Note(
    content={
        'decision_deadline': end_date.strftime('%Y/%m/%d'),
        'make_decisions_public': "No, decisions should NOT be revealed publicly when they are posted",
        'release_decisions_to_authors': 'Yes, decisions should be revealed when they are posted to the paper\'s authors',
        'release_decisions_to_reviewers': 'Yes, decisions should be immediately revealed to the paper\'s reviewers',
        'release_decisions_to_area_chairs': 'Yes, decisions should be immediately revealed to the paper\'s area chairs',
        'decisions_file': url
    },
    forum=request_form.forum,
    invitation=f'openreview.net/Support/-/Request{request_form.number}/Decision_Stage',
    readers=['TC/2024/Conference/Program_Chairs', 'openreview.net/Support'],
    replyto=request_form.forum,
    referent=request_form.forum,
    signatures=['~Program_Chair1'],
    writers=[]
))

helpers.await_queue()

invitation = client.get_invitation(f'openreview.net/Support/-/Request{request_form.number}/Post_Decision_Stage')
invitation.cdate = openreview.tools.datetime_millis(datetime.datetime.utcnow())
client.post_invitation(invitation)

Invitation(id = 'openreview.net/Support/-/Request1/Post_Decision_Stage',super = 'openreview.net/Support/-/Post_Decision_Stage',cdate = 1728075760188,ddate = None,duedate = None,expdate = None,readers = ['everyone'],nonreaders = None,writers = None,invitees = ['TC/2024/Conference/Program_Chairs', 'openreview.net/Support'],noninvitees = None,signatures = ['~Super_User1'],multiReply = True,taskCompletionCount = None,reply = {'readers': {'description': 'The users who will be allowed to read the above content.', 'values': ['TC/2024/Conference/Program_Chairs', 'openreview.net/Support']}, 'writers': {'values': []}, 'signatures': {'values-regex': '~.*'}, 'content': {'submission_readers': {'description': 'Please select who should have access to the submissions after the submission deadline. Note that program chairs and paper authors are always readers of submissions.', 'value-radio': ['All program committee (all reviewers, all area chairs, all senior area chairs if applicable)', 'All area chair

## Post Decision Stage

Sends the emails to notify authors, and releases papers publicly

In [40]:
# post a post decision stage note
short_name = 'TC 2024'
post_decision_stage_note = pc_client.post_note(openreview.Note(
    content={
        'reveal_authors': 'Reveal author identities of only accepted submissions to the public',
        'submission_readers': 'Everyone (submissions are public)',
        'home_page_tab_names': {
            'Accept (Oral)': 'Accept (Oral)',
            'Accept (Poster)': 'Accept (Poster)',
            'Reject': 'Submitted'
        },
        'send_decision_notifications': 'No, I will send the emails to the authors',
        'accept_(oral)_email_content': f'''Dear {{{{fullname}}}},

Thank you for submitting your paper, {{{{submission_title}}}}, to {short_name}. We are delighted to inform you that your submission has been accepted. Congratulations!
You can find the final reviews for your paper on the submission page in OpenReview at: {{{{forum_url}}}}

Best,
{short_name} Program Chairs
''',
        'accept_(poster)_email_content': f'''Dear {{{{fullname}}}},

Thank you for submitting your paper, {{{{submission_title}}}}, to {short_name}. We are delighted to inform you that your submission has been accepted. Congratulations!
You can find the final reviews for your paper on the submission page in OpenReview at: {{{{forum_url}}}}

Best,
{short_name} Program Chairs
''',
        'reject_email_content': f'''Dear {{{{fullname}}}},

Thank you for submitting your paper, {{{{submission_title}}}}, to {short_name}. We regret to inform you that your submission was not accepted.
You can find the final reviews for your paper on the submission page in OpenReview at: {{{{forum_url}}}}

Best,
{short_name} Program Chairs
'''
    },
    forum=request_form.forum,
    invitation=f'openreview.net/Support/-/Request{request_form.number}/Post_Decision_Stage',
    readers=['TC/2024/Conference/Program_Chairs', 'openreview.net/Support'],
    replyto=request_form.forum,
    referent=request_form.forum,
    signatures=['~Program_Chair1'],
    writers=[]
))
assert post_decision_stage_note
helpers.await_queue()

## Start Camera Ready Revision
from test_camera_ready

This allows users to edit their submission after recieving the decision and until due_date

In [28]:
# Post revision stage note
now = datetime.datetime.utcnow()
start_date = now - datetime.timedelta(days=2)
due_date = now + datetime.timedelta(days=5)
revision_stage_note = pc_client.post_note(openreview.Note(
    content={
        'submission_revision_name': 'Camera_Ready_Revision',
        'submission_revision_start_date': start_date.strftime('%Y/%m/%d'),
        'submission_revision_deadline': due_date.strftime('%Y/%m/%d'),
        'accepted_submissions_only': 'Enable revision for accepted submissions only',
        'submission_author_edition': 'Allow reorder of existing authors only',
        'submission_revision_additional_options': {
            "submission_type": {
                "value": {
                    "param": {
                        "type": "string",
                        "enum": [
                            "Regular Long Paper",
                            "Regular Short Paper"
                        ],
                        "input": "select"
                    }
                },
                "description": "Please enter the category under which the submission should be reviewed. This cannot be changed after the abstract submission deadline.",
                "order": 20
            }
        },
        'submission_revision_remove_options': ['keywords']
    },
    forum=request_form.forum,
    invitation=f'openreview.net/Support/-/Request{request_form.number}/Submission_Revision_Stage',
    readers=['TC/2024/Conference/Program_Chairs', 'openreview.net/Support'],
    referent=request_form.forum,
    replyto=request_form.forum,
    signatures=['~Program_Chair1'],
    writers=[]
))
helpers.await_queue()
